In [1]:
%load_ext autoreload
%autoreload 2

# PlaceRecognitionPipeline with SOC - ITLP dataset test

A module that implements a neural network algorithm for searching a database of places already visited by a vehicle for the most similar records using data from lidars and cameras and highlighting special elements of a three-dimensional scene (doors, buildings, street signs, etc.).

In [2]:
import copy

from pathlib import Path
from time import time

import faiss
import numpy as np
import open3d as o3d
import open3d.core as o3c
import pandas as pd
import torch
import torchshow as ts

from hydra.utils import instantiate
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation
from torch import Tensor
from torch.utils.data import DataLoader
from tqdm import tqdm

from geotransformer.utils.registration import compute_registration_error


from opr.datasets.itlp import ITLPCampus
from opr.pipelines.place_recognition import PlaceRecognitionPipeline
from opr.pipelines.registration import PointcloudRegistrationPipeline
from opr.pipelines.localization import LocalizationPipeline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.24 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [3]:
import os

os.environ["DISPLAY"] = ":1"

torch.cuda.synchronize()

In [4]:
def pose_to_matrix(pose):
    """From the 6D poses in the [tx ty tz qx qy qz qw] format to 4x4 pose matrices."""
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


# def compute_error(estimated_pose, gt_pose):
#     """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
#     estimated_pose = pose_to_matrix(estimated_pose)
#     gt_pose = pose_to_matrix(gt_pose)
#     return compute_registration_error(estimated_pose, gt_pose)

def compute_error(estimated_pose, gt_pose):
    """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return angle_error, dist_error


def draw_pc(pc: Tensor, color: str = "blue"):
    pc_o3d = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(pc))
    pcd = o3d.t.geometry.PointCloud(pc_o3d)
    if color == "blue":
        c = [0.0, 0.0, 1.0]
    elif color == "red":
        c = [1.0, 0.0, 0.0]
    else:
        c = [0.0, 1.0, 0.0]
    pcd = pcd.paint_uniform_color(c)
    o3d.visualization.draw_geometries(
        [pcd.to_legacy()],
    )


def invert_rigid_transformation_matrix(T: np.ndarray) -> np.ndarray:
    """
    Inverts a 4x4 rigid body transformation matrix.

    Args:
        T (np.ndarray): A 4x4 rigid body transformation matrix.

    Returns:
        np.ndarray: The inverted 4x4 rigid body transformation matrix.
    """
    assert T.shape == (4, 4), "Input matrix must be 4x4."

    R = T[:3, :3]
    t = T[:3, 3]

    R_inv = R.T
    t_inv = -R.T @ t

    T_inv = np.eye(4)
    T_inv[:3, :3] = R_inv
    T_inv[:3, 3] = t_inv

    return T_inv


def draw_pc_pair(
    pc_blue: Tensor, pc_blue_pose: np.ndarray | Tensor, pc_red: Tensor, pc_red_pose: np.ndarray | Tensor
):
    pc_blue_o3d = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(copy.deepcopy(pc_blue)))
    pc_red_o3d = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(copy.deepcopy(pc_red)))

    blue_pcd = o3d.t.geometry.PointCloud(pc_blue_o3d)
    blue_pcd_tmp = copy.deepcopy(blue_pcd)

    red_pcd = o3d.t.geometry.PointCloud(pc_red_o3d)
    red_pcd_tmp = copy.deepcopy(red_pcd)

    blue_pcd_tmp.voxel_down_sample(voxel_size=0.3)
    # blue_pcd_tmp.transform(pose_to_matrix(pc_blue_pose))
    blue_pcd_tmp = blue_pcd_tmp.paint_uniform_color([0.0, 0.0, 1.0])

    red_pcd_tmp.voxel_down_sample(voxel_size=0.3)
    red_pcd_tmp.transform(pose_to_matrix(pc_red_pose))
    red_pcd_tmp.transform(invert_rigid_transformation_matrix(pose_to_matrix(pc_blue_pose)))
    red_pcd_tmp = red_pcd_tmp.paint_uniform_color([1.0, 0.0, 0.0])
    o3d.visualization.draw_geometries(
        [blue_pcd_tmp.to_legacy(), red_pcd_tmp.to_legacy()],
    )


You can download the dataset:

- Kaggle:
  - [ITLP Campus Outdoor](https://www.kaggle.com/datasets/alexandermelekhin/itlp-campus-outdoor)
- Hugging Face:
  - [ITLP Campus Outdoor](https://huggingface.co/datasets/OPR-Project/ITLP-Campus-Outdoor)


In [ ]:
DATASET_ROOT = "/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor"

TRACK_LIST = sorted([str(subdir.name) for subdir in Path(DATASET_ROOT).iterdir() if subdir.is_dir()])
TRACK_LIST = TRACK_LIST[5:]

print("Test track list:")
print(TRACK_LIST)

Test track list:
['05_2023-08-15-day', '06_2023-08-18-night', '07_2023-10-04-day', '08_2023-10-11-night']


# Original version

In [6]:
SENSOR_SUITE = ["front_cam", "back_cam", "lidar"]

BATCH_SIZE = 4
NUM_WORKERS = 4
DEVICE = "cuda"

PR_MODEL_CONFIG_PATH = "../../configs/model/place_recognition/multimodal_with_soc_outdoor.yaml"
PR_WEIGHTS_PATH = "../../weights/place_recognition/multimodal_with_soc_outdoor_nclt.pth"


## Init models

In [7]:
pr_model_config = OmegaConf.load(PR_MODEL_CONFIG_PATH)
pr_model = instantiate(pr_model_config)
pr_model.load_state_dict(torch.load(PR_WEIGHTS_PATH))
pr_model = pr_model.to(DEVICE)
pr_model.eval();

sensors_cfg = OmegaConf.load("/home/docker_opr/OpenPlaceRecognition/configs/dataset/sensors_cfg/husky.yaml")
anno_cfg = OmegaConf.load("/home/docker_opr/OpenPlaceRecognition/configs/dataset/anno/oneformer.yaml")

## Calculate descriptors for databases

In [8]:
dataset = ITLPCampus(
    dataset_root=DATASET_ROOT,
    subset="test",
    csv_file="test.csv",
    sensors=SENSOR_SUITE,
    load_semantics=False,
    load_soc=True,
    sensors_cfg=sensors_cfg,
    anno=anno_cfg,
)
dataset.dataset_df = dataset.dataset_df[dataset.dataset_df["track"].isin(TRACK_LIST)]
dataset.dataset_df.reset_index(inplace=True)

no_masks = []
no_masks_filenames = []

for index, row in dataset.dataset_df.iterrows():
    mask_path = f"{DATASET_ROOT}/{row['track']}/masks/back_cam/{row['back_cam_ts']}.png"
    filename = f"{row['back_cam_ts']}.png"
    if not Path(mask_path).exists():
        no_masks.append(index)
        no_masks_filenames.append(filename)

dataset.dataset_df.drop(no_masks, inplace=True)
dataset.dataset_df.reset_index(inplace=True)

dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=dataset.collate_fn,
)


In [9]:
len(no_masks)

0

In [10]:
descriptors = []
with torch.no_grad():
    for batch in tqdm(dataloader):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        final_descriptor = pr_model(batch)["final_descriptor"]
        descriptors.append(final_descriptor.detach().cpu().numpy())

descriptors = np.concatenate(descriptors, axis=0)

100%|██████████| 153/153 [01:26<00:00,  1.77it/s]


### Saving database indexes

In [11]:
dataset_df = dataset.dataset_df

for track, indices in dataset_df.groupby("track").groups.items():
    track_descriptors = descriptors[indices]
    track_index = faiss.IndexFlatL2(track_descriptors.shape[1])
    track_index.add(track_descriptors)
    faiss.write_index(track_index, f"{DATASET_ROOT}/{track}/index.faiss")
    print(f"Saved index {DATASET_ROOT}/{track}/index.faiss")


Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/05_2023-08-15-day/index.faiss
Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/06_2023-08-18-night/index.faiss
Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/07_2023-10-04-day/index.faiss
Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/08_2023-10-11-night/index.faiss


## Test

In [12]:
RECALL_THRESHOLD = 25.0

test_csv = pd.read_csv(Path(DATASET_ROOT) / "test.csv", index_col=0)

all_pr_recalls = []

all_mean_pr_rotation_errors = []
all_mean_pr_translation_errors = []

all_median_pr_rotation_errors = []
all_median_pr_translation_errors = []

all_times = []

correct_examples = []  # the most representative correct pairs
pr_incorrect_examples = []  # the most representative incorrect pairs where place recognition failed

for db_track in TRACK_LIST:
    pr_pipe = PlaceRecognitionPipeline(
        database_dir=Path(DATASET_ROOT) / db_track,
        model=pr_model,
        model_weights_path=None,
        device=DEVICE,
    )
    for query_track in TRACK_LIST:
        if db_track == query_track:
            continue

        query_dataset = copy.deepcopy(dataset)
        query_dataset.dataset_df = query_dataset.dataset_df[query_dataset.dataset_df["track"] == query_track].reset_index(drop=True)
        query_df = query_dataset.dataset_df

        db_dataset = copy.deepcopy(dataset)
        db_dataset.dataset_df = db_dataset.dataset_df[db_dataset.dataset_df["track"] == db_track].reset_index(drop=True)
        db_df = db_dataset.dataset_df

        pr_pipe.database_df = db_df

        pr_matches = []
        pr_rotation_errors = []
        pr_translation_errors = []

        times = []

        for q_i, query in tqdm(enumerate(query_dataset)):
            query_pose = query_df.iloc[q_i][["tx", "ty", "tz", "qx", "qy", "qz", "qw"]].to_numpy()

            t = time()
            output = pr_pipe.infer(query)
            torch.cuda.current_stream().synchronize()
            times.append(time() - t)

            pr_rotation_error, pr_translation_error = compute_error(output["pose"], query_pose)

            pr_correct = pr_translation_error < RECALL_THRESHOLD

            pr_matches.append(pr_correct)
            pr_rotation_errors.append(pr_rotation_error)
            pr_translation_errors.append(pr_translation_error)

            if pr_correct and pr_translation_error < 5.0:  # close enough
                query["pose"] = query_pose
                db_match = db_dataset[output["idx"]]
                db_match["pose"] = output["pose"]
                correct_examples.append((query, db_match))

            if not pr_correct and pr_translation_error > 50.0:  # very far from GT
                query["pose"] = query_pose
                db_match = db_dataset[output["idx"]]
                db_match["pose"] = output["pose"]
                pr_incorrect_examples.append((query, db_match))

        all_pr_recalls.append(np.mean(pr_matches))

        all_mean_pr_rotation_errors.append(np.mean(pr_rotation_errors))
        all_mean_pr_translation_errors.append(np.mean(pr_translation_errors))
        all_median_pr_rotation_errors.append(np.median(pr_rotation_errors))
        all_median_pr_translation_errors.append(np.median(pr_translation_errors))

        all_times.extend(times[1:]) # drop the first iteration cause it is always slower

156it [00:48,  3.23it/s]
152it [00:46,  3.29it/s]
150it [00:46,  3.19it/s]
152it [00:47,  3.21it/s]
152it [00:44,  3.40it/s]
150it [00:51,  2.93it/s]
152it [00:44,  3.42it/s]
156it [00:49,  3.16it/s]
150it [00:50,  2.95it/s]
152it [00:50,  2.99it/s]
156it [01:04,  2.43it/s]
152it [00:47,  3.19it/s]


In [13]:
len(correct_examples), len(pr_incorrect_examples), #len(reg_incorrect_examples)

(1280, 47)

In [14]:
results_str = f"""Average PR Recall@1:   {np.mean(all_pr_recalls)*100:.2f}

Average Mean RRE PR:   {np.mean(all_mean_pr_rotation_errors):.2f}
Average Mean RTE PR:   {np.mean(all_mean_pr_translation_errors):.2f}
Average Median RRE PR:   {np.mean(all_median_pr_rotation_errors):.2f}
Average Median RTE PR:   {np.mean(all_median_pr_translation_errors):.2f}

Mean inference time:     {np.mean(all_times)*1000:.2f} ms
"""

In [15]:
print(results_str)

Average PR Recall@1:   96.39

Average Mean RRE PR:   10.07
Average Mean RTE PR:   6.48
Average Median RRE PR:   5.13
Average Median RTE PR:   3.38

Mean inference time:     38.63 ms



In [ ]:
correct_example = correct_examples[100]

query_sample, db_match_sample = correct_example

ts.show([
    query_sample["image_front_cam"], query_sample["image_back_cam"],
])
ts.show([
    db_match_sample["image_front_cam"], db_match_sample["image_back_cam"],
])
print(f"PR pose error: {compute_error(db_match_sample['pose'], query_sample['pose'])}")

In [ ]:
pr_incorrect_example = pr_incorrect_examples[20]

query_sample, db_match_sample = pr_incorrect_example

ts.show([
    query_sample["image_front_cam"], query_sample["image_back_cam"],
])
ts.show([
    db_match_sample["image_front_cam"], db_match_sample["image_back_cam"],
])
print(f"PR pose error: {compute_error(db_match_sample['pose'], query_sample['pose'])}")

# Improved version

In [18]:
SENSOR_SUITE = ["front_cam", "back_cam", "lidar"]

BATCH_SIZE = 4
NUM_WORKERS = 4
DEVICE = "cuda"

PR_MODEL_CONFIG_PATH = "../../configs/model/place_recognition/multimodal_with_soc_outdoor.yaml"
PR_WEIGHTS_PATH = "../../weights/place_recognition/multimodal_with_soc_outdoor_itlp-finetune.pth"

REGISTRATION_MODEL_CONFIG_PATH = "../../configs/model/registration/hregnet_light_feats.yaml"
REGISTRATION_WEIGHTS_PATH = "../../weights/registration/hregnet_light_feats_nuscenes.pth"

## Init models

In [19]:
pr_model_config = OmegaConf.load(PR_MODEL_CONFIG_PATH)
pr_model = instantiate(pr_model_config)
pr_model.load_state_dict(torch.load(PR_WEIGHTS_PATH))
pr_model = pr_model.to(DEVICE)
pr_model.eval();

reg_model_config = OmegaConf.load(REGISTRATION_MODEL_CONFIG_PATH)
reg_model = instantiate(reg_model_config)
reg_model.load_state_dict(torch.load(REGISTRATION_WEIGHTS_PATH))
reg_model = reg_model.to(DEVICE)
reg_model.eval();

## Calculate descriptors for databases

In [20]:
sensors_cfg = OmegaConf.load("/home/docker_opr/OpenPlaceRecognition/configs/dataset/sensors_cfg/husky.yaml")
anno_cfg = OmegaConf.load("/home/docker_opr/OpenPlaceRecognition/configs/dataset/anno/oneformer.yaml")

dataset = ITLPCampus(
    dataset_root=DATASET_ROOT,
    subset="test",
    csv_file="test.csv",
    sensors=SENSOR_SUITE,
    load_semantics=False,
    load_soc=True,
    sensors_cfg=sensors_cfg,
    anno=anno_cfg,
)

dataset.dataset_df = dataset.dataset_df[dataset.dataset_df["track"].isin(TRACK_LIST)]
dataset.dataset_df.reset_index(inplace=True)

dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=dataset.collate_fn,
)


In [21]:
descriptors = []
with torch.no_grad():
    for batch in tqdm(dataloader):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        final_descriptor = pr_model(batch)["final_descriptor"]
        descriptors.append(final_descriptor.detach().cpu().numpy())

descriptors = np.concatenate(descriptors, axis=0)

100%|██████████| 153/153 [01:06<00:00,  2.32it/s]


### Saving database indexes

In [22]:
dataset_df = dataset.dataset_df

for track, indices in dataset_df.groupby("track").groups.items():
    track_descriptors = descriptors[indices]
    track_index = faiss.IndexFlatL2(track_descriptors.shape[1])
    track_index.add(track_descriptors)
    faiss.write_index(track_index, f"{DATASET_ROOT}/{track}/index.faiss")
    print(f"Saved index {DATASET_ROOT}/{track}/index.faiss")


Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/05_2023-08-15-day/index.faiss
Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/06_2023-08-18-night/index.faiss
Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/07_2023-10-04-day/index.faiss
Saved index /home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/08_2023-10-11-night/index.faiss


## Test

In [23]:
RECALL_THRESHOLD = 25.0

test_csv = pd.read_csv(Path(DATASET_ROOT) / "test.csv", index_col=0)

all_pr_recalls = []
all_reg_recalls = []  # it is recall after registration (if estimated pose within RECALL_THRESHOLD), do not confuse with registration recall

all_mean_pr_rotation_errors = []
all_mean_pr_translation_errors = []

all_median_pr_rotation_errors = []
all_median_pr_translation_errors = []

all_mean_reg_rotation_errors = []
all_mean_reg_translation_errors = []

all_median_reg_rotation_errors = []
all_median_reg_translation_errors = []

all_times = []

correct_examples = []  # the most representative correct pairs
pr_incorrect_examples = []  # the most representative incorrect pairs where place recognition failed
reg_incorrect_examples = []  # the most representative incorrect pairs where registration failed

for db_track in TRACK_LIST:
    pr_pipe = PlaceRecognitionPipeline(
        database_dir=Path(DATASET_ROOT) / db_track,
        model=pr_model,
        model_weights_path=PR_WEIGHTS_PATH,
        device=DEVICE,
    )
    for query_track in TRACK_LIST:
        if db_track == query_track:
            continue

        reg_pipe = PointcloudRegistrationPipeline(
            model=reg_model,
            model_weights_path=REGISTRATION_WEIGHTS_PATH,
            device=DEVICE,
            voxel_downsample_size=0.3,
            num_points_downsample=8192,
        )
        loc_pipe = LocalizationPipeline(
            place_recognition_pipeline=pr_pipe,
            registration_pipeline=reg_pipe,
            precomputed_reg_feats=True,
            pointclouds_subdir="lidar",
        )

        query_dataset = copy.deepcopy(dataset)
        query_dataset.dataset_df = query_dataset.dataset_df[query_dataset.dataset_df["track"] == query_track].reset_index(drop=True)
        query_df = query_dataset.dataset_df

        db_dataset = copy.deepcopy(dataset)
        db_dataset.dataset_df = db_dataset.dataset_df[db_dataset.dataset_df["track"] == db_track].reset_index(drop=True)
        db_df = db_dataset.dataset_df

        loc_pipe.pr_pipe.database_df = db_df
        loc_pipe.database_df = db_df


        pr_matches = []
        pr_rotation_errors = []
        pr_translation_errors = []

        reg_matches = []
        reg_rotation_errors = []
        reg_translation_errors = []

        times = []

        for q_i, query in tqdm(enumerate(query_dataset)):
            query_pose = query_df.iloc[q_i][["tx", "ty", "tz", "qx", "qy", "qz", "qw"]].to_numpy()

            t = time()
            output = loc_pipe.infer(query)
            torch.cuda.current_stream().synchronize()
            times.append(time() - t)

            pr_rotation_error, pr_translation_error = compute_error(output["db_match_pose"], query_pose)
            reg_rotation_error, reg_translation_error = compute_error(output["estimated_pose"], query_pose)

            pr_correct = pr_translation_error < RECALL_THRESHOLD
            reg_correct = reg_translation_error < RECALL_THRESHOLD

            pr_matches.append(pr_correct)
            pr_rotation_errors.append(pr_rotation_error)
            pr_translation_errors.append(pr_translation_error)

            reg_matches.append(reg_correct)
            reg_rotation_errors.append(reg_rotation_error)
            reg_translation_errors.append(reg_translation_error)

            if pr_correct and reg_correct \
                and reg_rotation_error < pr_rotation_error and reg_translation_error < pr_translation_error \
                and reg_rotation_error < 3.0 and reg_translation_error < 1.0:
                query["pose"] = query_pose
                db_match = db_dataset[output["db_match_idx"]]
                db_match["pose"] = output["db_match_pose"]
                correct_examples.append((query, db_match, output["estimated_pose"]))

            if pr_correct and not reg_correct:
                query["pose"] = query_pose
                db_match = db_dataset[output["db_match_idx"]]
                db_match["pose"] = output["db_match_pose"]
                reg_incorrect_examples.append((query, db_match, output["estimated_pose"]))

            if not pr_correct and pr_translation_error > 50.0:
                query["pose"] = query_pose
                db_match = db_dataset[output["db_match_idx"]]
                db_match["pose"] = output["db_match_pose"]
                pr_incorrect_examples.append((query, db_match, output["estimated_pose"]))

        all_pr_recalls.append(np.mean(pr_matches))
        all_reg_recalls.append(np.mean(reg_matches))

        all_mean_pr_rotation_errors.append(np.mean(pr_rotation_errors))
        all_mean_pr_translation_errors.append(np.mean(pr_translation_errors))
        all_median_pr_rotation_errors.append(np.median(pr_rotation_errors))
        all_median_pr_translation_errors.append(np.median(pr_translation_errors))

        all_mean_reg_rotation_errors.append(np.mean(reg_rotation_errors))
        all_mean_reg_translation_errors.append(np.mean(reg_translation_errors))
        all_median_reg_rotation_errors.append(np.median(reg_rotation_errors))
        all_median_reg_translation_errors.append(np.median(reg_translation_errors))
        all_times.extend(times[1:]) # drop the first iteration cause it is always slower

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/docker_opr/.local/lib/python3.10/site-packages/hregnet/utils.py:24: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  output = torch.cuda.IntTensor(B, npoint)
156it [00:38,  4.02it/s]
152it [00:37,  4.02it/s]
150it [00:35,  4.25it/s]
152it [00:37,  4.10it/s]
152it [00:34,  4.35it/s]
150it [00:36,  4.05it/s]
152it [00:36,  4.17it/s]
156it [00:36,  4.25it/s]
150it [00:35,  4.18it/s]
152

In [24]:
len(correct_examples), len(pr_incorrect_examples), len(reg_incorrect_examples)

(35, 5, 1)

In [25]:
results_str = f"""Average REG Recall@1:  {np.mean(all_reg_recalls)*100:.2f}

Average Mean RRE REG:  {np.mean(all_mean_reg_rotation_errors):.2f}
Average Mean RTE REG:  {np.mean(all_mean_reg_translation_errors):.2f}
Average Median RRE REG:  {np.mean(all_median_reg_rotation_errors):.2f}
Average Median RTE REG:  {np.mean(all_median_reg_translation_errors):.2f}

Mean inference time (PR + REG):     {np.mean(all_times)*1000:.2f} ms
"""

In [26]:
print(results_str)

Average REG Recall@1:  99.62

Average Mean RRE REG:  9.24
Average Mean RTE REG:  3.58
Average Median RRE REG:  4.90
Average Median RTE REG:  2.56

Mean inference time (PR + REG):     71.11 ms



In [ ]:
correct_example = correct_examples[20]

query_sample, db_match_sample, estimated_pose = correct_example

draw_pc_pair(
    query_sample["pointcloud_lidar_coords"],
    estimated_pose,
    db_match_sample["pointcloud_lidar_coords"],
    db_match_sample["pose"],
)

ts.show([
    query_sample["image_front_cam"], query_sample["image_back_cam"],
])
ts.show([
    db_match_sample["image_front_cam"], db_match_sample["image_back_cam"],
])
print(f"PR pose error: {compute_error(db_match_sample['pose'], query_sample['pose'])}")
print(f"REG pose error: {compute_error(estimated_pose, query_sample['pose'])}")

In [ ]:
pr_incorrect_example = pr_incorrect_examples[0]

query_sample, db_match_sample, estimated_pose = correct_example

ts.show([
    query_sample["image_front_cam"], query_sample["image_back_cam"],
])
ts.show([
    db_match_sample["image_front_cam"], db_match_sample["image_back_cam"],
])
print(f"PR pose error: {compute_error(db_match_sample['pose'], query_sample['pose'])}")
print(f"REG pose error: {compute_error(estimated_pose, query_sample['pose'])}")

In [ ]:
reg_incorrect_example = reg_incorrect_examples[0]

query_sample, db_match_sample, estimated_pose = reg_incorrect_example

ts.show([
    query_sample["image_Cam5"], query_sample["image_Cam2"],
])
ts.show([
    db_match_sample["image_Cam5"], db_match_sample["image_Cam2"],
])
print(f"PR pose error: {compute_error(db_match_sample['pose'], query_sample['pose'])}")
print(f"REG pose error: {compute_error(estimated_pose, query_sample['pose'])}")